In [1]:
# Step 1: project folders + DuckDB connection smoke test
from pathlib import Path
import platform, sys
import duckdb

# ---- EDIT if you want a different project name ----
PROJECT_NAME = "kwh_prediction"

# Project root relative to current notebook
ROOT = Path.cwd() / PROJECT_NAME
PATHS = {
    "data_raw": ROOT / "data" / "raw",
    "data_clean": ROOT / "data" / "clean",
    "data_feature": ROOT / "data" / "feature",
    "models": ROOT / "models",
    "inference": ROOT / "inference",
    "reports": ROOT / "reports",
    "logs": ROOT / "logs",
    "config_dir": ROOT / "config",
    "src": ROOT / "src",
}

# Create folders
for p in PATHS.values():
    p.mkdir(parents=True, exist_ok=True)

# DuckDB file (as you said)
DUCKDB_DB = (Path.cwd() / ".." / "data" / "warehouse.duckdb").resolve()
print("Python:", platform.python_version())
print("Notebook cwd:", Path.cwd())
print("Project root:", ROOT)
print("DuckDB path:", DUCKDB_DB)

# Smoke test: connect and list tables (schema 'main')
conn = duckdb.connect(str(DUCKDB_DB))
tables = conn.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
    ORDER BY 1
""").fetchall()
print(f"Found {len(tables)} tables in DuckDB.")
print("First few:", [t[0] for t in tables[:10]])
conn.close()

# Optional: quick tree printer for sanity
def print_tree(base: Path, max_depth: int = 2, prefix: str = "") -> None:
    def _walk(path: Path, depth: int, pref: str):
        if depth > max_depth:
            return
        entries = sorted(path.iterdir(), key=lambda p: (p.is_file(), p.name.lower()))
        for i, entry in enumerate(entries):
            connector = "└── " if i == len(entries)-1 else "├── "
            print(pref + connector + entry.name + ("" if entry.is_file() else "/"))
            if entry.is_dir():
                _walk(entry, depth+1, pref + ("    " if i == len(entries)-1 else "│   "))
    print(base.name + "/"); _walk(base, 1, "")

print_tree(ROOT, max_depth=2)


Python: 3.13.5
Notebook cwd: C:\Users\Asus\PycharmProjects\NEDCO_data_viz_app\notebooks
Project root: C:\Users\Asus\PycharmProjects\NEDCO_data_viz_app\notebooks\kwh_prediction
DuckDB path: C:\Users\Asus\PycharmProjects\NEDCO_data_viz_app\data\warehouse.duckdb
Found 0 tables in DuckDB.
First few: []
kwh_prediction/
├── config/
├── data/
│   ├── clean/
│   ├── feature/
│   └── raw/
├── inference/
├── logs/
├── models/
├── reports/
└── src/


In [2]:
# Step 2: list all tables (any schema) and quick counts
import duckdb

conn = duckdb.connect(str(DUCKDB_DB))

tables_info = conn.execute("""
SELECT table_schema, table_name, table_type
FROM information_schema.tables
ORDER BY table_schema, table_name
""").fetchdf()

print(f"Found {len(tables_info)} tables/views total.")
display(tables_info.head(20))


Found 1 tables/views total.


,table_schema,table_name,table_type
0,prod,sales,BASE TABLE


In [3]:
# Step 3: inspect prod.sales columns + a few sample rows
import duckdb
import pandas as pd

conn = duckdb.connect(str(DUCKDB_DB))

# Columns & types
cols = conn.execute("""
    PRAGMA table_info('prod.sales')
""").fetchdf()

print("Columns in prod.sales:")
display(cols)

# Quick sample
sample = conn.execute("""
    SELECT * FROM prod.sales
    LIMIT 5
""").fetchdf()
print("Sample rows:")
display(sample)

conn.close()


Columns in prod.sales:


,cid,name,type,notnull,dflt_value,pk
0,0,chargedate,DATE,False,None,False
1,1,meterid,BIGINT,False,None,False
2,2,chargedate_str,VARCHAR,False,None,False
3,3,loc,VARCHAR,False,None,False
4,4,res,VARCHAR,False,None,False
5,5,month,VARCHAR,False,None,False
6,6,month_str,VARCHAR,False,None,False
7,7,kwh,DOUBLE,False,None,False
8,8,year,BIGINT,False,None,False
9,9,ghc,DOUBLE,False,None,False


Sample rows:


,chargedate,meterid,chargedate_str,loc,res,month,month_str,kwh,year,ghc,paymoney
0,2019-02-15,1,15-Feb-2019,Techiman [13],N-Resid [0],Feb-19,01-Feb-2019,14.2,2019,23.1705,140.0
1,2019-03-11,1,11-Mar-2019,Techiman [13],N-Resid [0],Mar-19,01-Mar-2019,57.4,2019,38.8886,40.0
2,2019-04-20,1,20-Apr-2019,Techiman [13],N-Resid [0],Apr-19,01-Apr-2019,57.4,2019,38.8886,20.0
3,2019-05-28,1,28-May-2019,Techiman [13],N-Resid [0],May-19,01-May-2019,57.4,2019,38.8886,20.0
4,2019-06-25,1,25-Jun-2019,Techiman [13],N-Resid [0],Jun-19,01-Jun-2019,57.4,2019,38.8886,20.0


In [4]:
# Step 4: write CONFIG for DuckDB source prod.sales
from pathlib import Path
import json
try:
    import yaml
except Exception:
    yaml = None

# Reuse PROJECT_NAME, PATHS, DUCKDB_DB from earlier cells
CONFIG = {
    "project_name": PROJECT_NAME,
    "io": {
        "engine": "duckdb",
        "duckdb_path": str(DUCKDB_DB),
        "table": "prod.sales",        # discovered in Step 2
        "sql": None                   # optional override query (leave None for whole table)
    },
    "data": {
        "id_col": "meterid",
        "date_col": "chargedate",     # prefer DATE column
        "month_col": "month",         # string month exists too; we’ll parse later if needed
        "target_col": "kwh",
        "location_col": "loc",
        "residence_col": "res",
        "currency_cols": ["ghc", "paymoney"],
        "expected_frequency": "M"
    },
    "time": {
        "unreliable_after": "2020-09-30",  # exclusive
        "tz": "UTC"
    },
    "paths": {k: str(v) for k, v in PATHS.items()},
    "random_seed": 42,
    "evaluation_metrics": ["rmse", "mae", "mape", "r2"]
}

# Write config files
config_dir = Path(PATHS["config_dir"])
config_dir.mkdir(parents=True, exist_ok=True)

with open(config_dir / "config.json", "w", encoding="utf-8") as f:
    json.dump(CONFIG, f, indent=2)
print("Wrote", (config_dir / "config.json").as_posix())

if yaml is not None:
    with open(config_dir / "config.yaml", "w", encoding="utf-8") as f:
        yaml.safe_dump(CONFIG, f, sort_keys=False)
    print("Wrote", (config_dir / "config.yaml").as_posix())
else:
    print("PyYAML not installed; wrote JSON only.")

# Quick preview
print(json.dumps(CONFIG, indent=2)[:800] + "\n...")


Wrote C:/Users/Asus/PycharmProjects/NEDCO_data_viz_app/notebooks/kwh_prediction/config/config.json
Wrote C:/Users/Asus/PycharmProjects/NEDCO_data_viz_app/notebooks/kwh_prediction/config/config.yaml
{
  "project_name": "kwh_prediction",
  "io": {
    "engine": "duckdb",
    "duckdb_path": "C:\\Users\\Asus\\PycharmProjects\\NEDCO_data_viz_app\\data\\warehouse.duckdb",
    "table": "prod.sales",
    "sql": null
  },
  "data": {
    "id_col": "meterid",
    "date_col": "chargedate",
    "month_col": "month",
    "target_col": "kwh",
    "location_col": "loc",
    "residence_col": "res",
    "currency_cols": [
      "ghc",
      "paymoney"
    ],
    "expected_frequency": "M"
  },
  "time": {
    "unreliable_after": "2020-09-30",
    "tz": "UTC"
  },
  "paths": {
    "data_raw": "C:\\Users\\Asus\\PycharmProjects\\NEDCO_data_viz_app\\notebooks\\kwh_prediction\\data\\raw",
    "data_clean": "C:\\Users\\Asus\\PycharmProjects\\NEDCO_data_viz_app\\notebooks\\kwh_prediction\\data\\clean",
    "
.

In [5]:
# Step 5: reproducibility + logging helpers
import random, numpy as np, logging, sys
from pathlib import Path

def set_seed(seed: int = 42):
    """Set global random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    try:
        import torch
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
    except Exception:
        pass  # torch optional

def get_logger(name="kwh_project", level=logging.INFO):
    """Return a logger that writes both to console and file."""
    logger = logging.getLogger(name)
    logger.setLevel(level)
    logger.propagate = False  # avoid duplicate logs in notebooks

    # clear old handlers
    for h in list(logger.handlers):
        logger.removeHandler(h)

    ch = logging.StreamHandler(sys.stdout)
    ch.setLevel(level)
    ch.setFormatter(logging.Formatter("[%(levelname)s] %(message)s"))

    log_path = Path(PATHS["logs"]) / "project.log"
    fh = logging.FileHandler(log_path)
    fh.setLevel(level)
    fh.setFormatter(logging.Formatter("%(asctime)s %(levelname)s: %(message)s"))

    logger.addHandler(ch)
    logger.addHandler(fh)
    return logger

# Initialize
set_seed(CONFIG["random_seed"])
logger = get_logger()
logger.info("Random seed and logger initialized.")


[INFO] Random seed and logger initialized.


In [6]:
# Step 6: environment check (Python + key libraries)
import platform

def env_report():
    versions = {"python": platform.python_version()}
    def get_ver(name):
        try:
            mod = __import__(name)
            return getattr(mod, "__version__", "installed")
        except Exception:
            return None

    libs = ["duckdb", "pandas", "numpy", "scikit_learn", "xgboost", "lightgbm", "matplotlib"]
    for lib in libs:
        name = "sklearn" if lib == "scikit_learn" else lib
        versions[lib] = get_ver(name)

    print("Environment versions:")
    for k, v in versions.items():
        print(f"  {k:15s}: {v if v else '— not installed —'}")

env_report()


Environment versions:
  python         : 3.13.5
  duckdb         : 1.4.0
  pandas         : 2.3.2
  numpy          : 2.3.3
  scikit_learn   : 1.7.2
  xgboost        : — not installed —
  lightgbm       : 4.6.0
  matplotlib     : 3.10.6


In [7]:
# Step 7: create src/utils.py with helper functions
from pathlib import Path

UTILS_CODE = """
import pandas as pd
import time
from contextlib import contextmanager
from pathlib import Path

def memory_usage_mb(df: pd.DataFrame) -> float:
    '''Return approximate DataFrame memory usage (MB).'''
    return float(df.memory_usage(deep=True).sum()) / (1024 ** 2)

@contextmanager
def timer(name: str):
    t0 = time.time()
    yield
    print(f"[{name}] done in {time.time() - t0:.2f}s")

def safe_to_datetime(series, dayfirst=False):
    '''Coerce to datetime, return NaT on failure.'''
    return pd.to_datetime(series, errors='coerce', dayfirst=dayfirst)

def add_season_column(df: pd.DataFrame, month_col='month', out_col='season'):
    '''Add simple season labels based on month number.'''
    season_map = {
        12:'winter',1:'winter',2:'winter',
        3:'spring',4:'spring',5:'spring',
        6:'summer',7:'summer',8:'summer',
        9:'fall',10:'fall',11:'fall'
    }
    df[out_col] = df[month_col].map(season_map)
    return df
"""

SRC_DIR = Path(PATHS["src"])
SRC_DIR.mkdir(parents=True, exist_ok=True)
with open(SRC_DIR / "utils.py", "w", encoding="utf-8") as f:
    f.write(UTILS_CODE.strip())

print("Wrote:", (SRC_DIR / "utils.py").as_posix())


Wrote: C:/Users/Asus/PycharmProjects/NEDCO_data_viz_app/notebooks/kwh_prediction/src/utils.py


In [8]:
# Step 8: create simple data dictionary template
import csv
from pathlib import Path

docs_dir = Path(PATHS["reports"]) / "docs"
docs_dir.mkdir(parents=True, exist_ok=True)
dd_path = docs_dir / "data_dictionary_template.csv"

rows = [
    ["column", "description", "type", "allowed_values/examples"],
    ["meterid", "Unique meter identifier", "categorical", ""],
    ["chargedate", "Date of energy charge", "date", "YYYY-MM-DD"],
    ["loc", "Location (e.g., Techiman)", "categorical", ""],
    ["res", "Residence type (e.g., N-Resid [0])", "categorical", ""],
    ["kwh", "Energy usage (target)", "numeric", ">=0"],
    ["ghc", "Energy cost", "numeric", ""],
    ["paymoney", "Payment made", "numeric", ""],
]

with open(dd_path, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerows(rows)

print("Wrote:", dd_path.as_posix())
print("\n✅ Setup notebook finished. You’re ready to start 01-EDA-and-Data-Cleaning.ipynb next.")


Wrote: C:/Users/Asus/PycharmProjects/NEDCO_data_viz_app/notebooks/kwh_prediction/reports/docs/data_dictionary_template.csv

✅ Setup notebook finished. You’re ready to start 01-EDA-and-Data-Cleaning.ipynb next.
